<a href="https://www.kaggle.com/code/mdshahnewazibrahim/ai-news-summarizer-notebook?scriptVersionId=278692417" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🌐 AI News Summarizer (Kaggle Capstone Project)

This notebook demonstrates an AI-powered news summarization pipeline using **Google Gemini 2.5 Flash**.
The goal is to:

- Fetch real news articles from the web (public URLs)
- Clean and extract the main article text
- Send the content to Gemini 2.5 Flash
- Receive a **structured JSON-style summary** that includes:
  - Main summary
  - Bullet points
  - Category
  - Entities
  - Sentiment
  - Impact
- Optionally build a **daily digest** from multiple articles
- Run a simple **evaluation** against human-written summaries

---

## 1. Problem Statement

Modern news consumption suffers from information overload:
- Long articles are time-consuming to read
- Readers want quick, clear summaries
- Bangla-friendly summaries for international news are rare

This project builds an AI News Summarizer that converts raw news URLs into concise,
human-friendly summaries in multiple styles.

---

## 2. High-Level Architecture

```text
News URL
   │
   ▼
[ Scraper & Cleaner ]   →  Extract plain text from HTML
   │
   ▼
[ Prompt Builder ]       →  Add instructions + article
   │
   ▼
[ Gemini 2.5 Flash ]     →  Generate JSON-style summary
   │
   ▼
[ Output Formatter ]     →  Pretty print results / daily digest
```

---


In [1]:
# ==================================
# 1. Imports & Configuration
# ==================================

import json
import re
import textwrap
from typing import List, Dict, Any, Optional

import requests
from bs4 import BeautifulSoup

import google.generativeai as gen
from kaggle_secrets import UserSecretsClient

# ----------------------------------
# Load Gemini API Key from Kaggle Secrets
# ----------------------------------

secrets = UserSecretsClient()
GOOGLE_API_KEY = secrets.get_secret("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("❌ GOOGLE_API_KEY not found in Kaggle secrets. Please set it before running.")

gen.configure(api_key=GOOGLE_API_KEY)

MODEL_NAME = "gemini-2.5-flash"
model = gen.GenerativeModel(MODEL_NAME)

# ----------------------------------
# Global Configuration
# ----------------------------------

MAX_ARTICLE_CHARS = 4000   # Trim long articles before sending to the model
MIN_PARAGRAPH_LEN = 40     # Skip very short <p> tags (likely noise)
MIN_ARTICLE_LEN = 200      # Minimum length for summarization

LANGUAGE_MODES = {
    "bn_simple": "Bangla, very simple, conversational tone.",
    "bn_formal": "Bangla, formal news style.",
    "en_simple": "English, simple beginner-friendly style.",
    "en_formal": "English, formal journalistic style.",
}

print("✅ Configuration loaded. Gemini model initialized.")


✅ Configuration loaded. Gemini model initialized.


In [2]:
# ==================================
# 2. Scraper & Helper Functions
# ==================================

def fetch_article_from_url(url: str) -> tuple[str, str]:
    """Fetch the title and main article text from a news URL."""
    print(f"🌐 Fetching article from: {url}")
    try:
        resp = requests.get(url, timeout=20)
        resp.raise_for_status()
    except Exception as e:
        print("❌ Request failed:", e)
        return "Untitled", ""

    soup = BeautifulSoup(resp.text, "html.parser")

    # Title
    title = soup.title.get_text(strip=True) if soup.title else "Untitled"

    # Collect paragraph text
    paragraphs = []
    for p in soup.find_all("p"):
        text = p.get_text(" ", strip=True)
        if text and len(text) >= MIN_PARAGRAPH_LEN:
            paragraphs.append(text)

    article_text = "\n".join(paragraphs).strip()
    print(f"✅ Extracted {len(article_text.split())} words from article.")
    return title, article_text


def clean_json_string(s: str) -> str:
    """Remove ```json ... ``` wrappers if the model returns code fences."""
    s = s.strip()
    if s.startswith("```"):
        # remove leading ```json or ```
        s = re.sub(r"^```(?:json)?", "", s, flags=re.IGNORECASE).strip()
        # remove trailing ```
        if s.endswith("```"):
            s = s[:-3].strip()
    return s


def safe_load_json(s: str) -> Dict[str, Any]:
    """Try to parse JSON from a model output string safely.
    If parsing fails, return a dict with the raw text under 'summary'.
    """
    s = clean_json_string(s)
    try:
        return json.loads(s)
    except Exception:
        # Fallback: treat the whole string as plain summary text
        return {"summary": s}

print("✅ Helper functions ready.")


✅ Helper functions ready.


In [3]:
# ==================================
# 3. Core NewsSummarizer Class (Gemini 2.5 Flash)
# ==================================

class NewsSummarizer:
    """High-level wrapper around Gemini 2.5 Flash for news summarization."""

    def __init__(self, model: Any):
        self.model = model

    def _build_instruction(self, mode: str) -> str:
        """Build the instruction text for the model."""
        style = LANGUAGE_MODES.get(mode, LANGUAGE_MODES["en_simple"])

        instruction = f"""
You are an expert news summarization assistant.

You will receive the raw text of a news article and the source URL.
Your task is to produce a JSON object with the following exact keys:

- "language": string (e.g., "bn" or "en")
- "summary": string (2–4 short paragraphs)
- "bullets": array of strings (5–7 key bullet points)
- "category": string (e.g., "Politics", "Technology", "Economy", "Health", "Sports", "Environment", "Other")
- "entities": array of strings (important people, places, organizations, countries)
- "sentiment": string ("Positive", "Negative", "Neutral", or "Mixed")
- "impact": string (who is affected and how, in 1–3 sentences)

Write the summary and bullets in this style:
{style}

Return only valid JSON, with no extra commentary, no backticks, and no markdown.
"""
        return textwrap.dedent(instruction).strip()

    def _build_prompt(self, article_text: str, url: Optional[str] = None) -> str:
        """Build the user-facing part of the prompt from article text + URL."""
        article_trimmed = article_text[:MAX_ARTICLE_CHARS]

        prompt = f"{article_trimmed}\n\nSource URL: {url or 'N/A'}"
        return prompt

    def summarize_article(
        self,
        article_text: str,
        url: Optional[str] = None,
        mode: str = "bn_simple",
    ) -> Dict[str, Any]:
        """Summarize raw article text using Gemini and return a structured dict."""
        if not article_text or len(article_text) < MIN_ARTICLE_LEN:
            return {
                "language": "unknown",
                "summary": "Article text is too short to summarize.",
                "bullets": [],
                "category": "Other",
                "entities": [],
                "sentiment": "Neutral",
                "impact": "",
            }

        instruction = self._build_instruction(mode)
        user_prompt = self._build_prompt(article_text, url=url)

        # Single string prompt (no roles / system_instruction to avoid API issues)
        full_prompt = instruction + "\n\nArticle:\n\n" + user_prompt

        try:
            response = self.model.generate_content(full_prompt)
            raw_text = response.text or ""
            data = safe_load_json(raw_text)

            # Ensure all expected keys exist
            data.setdefault("language", "unknown")
            data.setdefault("summary", "")
            data.setdefault("bullets", [])
            data.setdefault("category", "Other")
            data.setdefault("entities", [])
            data.setdefault("sentiment", "Neutral")
            data.setdefault("impact", "")
            return data
        except Exception as e:
            print("❌ Gemini error:", e)
            return {
                "language": "unknown",
                "summary": f"Model error: {e}",
                "bullets": [],
                "category": "Other",
                "entities": [],
                "sentiment": "Neutral",
                "impact": "",
            }

    def summarize_url(self, url: str, mode: str = "bn_simple") -> Dict[str, Any]:
        """End-to-end: URL → fetch article → summarize with Gemini."""
        title, article_text = fetch_article_from_url(url)
        result = self.summarize_article(article_text, url=url, mode=mode)
        result["title"] = title
        result["url"] = url
        return result


# create a global summarizer instance
summarizer = NewsSummarizer(model)
print("✅ NewsSummarizer is ready.")



✅ NewsSummarizer is ready.


In [4]:
# ==================================
# 4. Wrapper & Pretty Printer
# ==================================

def summarize_news_url(url: str, mode: str = "bn_simple") -> Dict[str, Any]:
    """Simple function to summarize a single news URL."""
    return summarizer.summarize_url(url, mode=mode)


def print_summary(result: Dict[str, Any]) -> None:
    """Pretty-print a summary result dict."""
    print("=" * 80)
    print("📰 TITLE:", result.get("title", "(no title)"))
    print("🔗 URL   :", result.get("url", ""))
    print("🏷️  Category:", result.get("category", ""))
    print("😊 Sentiment:", result.get("sentiment", ""))
    print("-" * 80)
    print("📄 SUMMARY:\n")
    print(result.get("summary", ""))
    print("\n• Bullet Points:")
    for i, b in enumerate(result.get("bullets", []), start=1):
        print(f"  {i}. {b}")
    if result.get("entities"):
        print("\n🔑 Entities:")
        print(", ".join(result["entities"]))
    if result.get("impact"):
        print("\n📌 Impact:")
        print(result["impact"])
    print("=" * 80)

print("✅ Wrapper and printer ready.")


✅ Wrapper and printer ready.


In [5]:
# ==================================
# 5. Single Article Demo (you can change the URL)
# ==================================

demo_url = "https://www.bbc.com/news/articles/c891jp9j79do"  # example URL

demo_result = summarize_news_url(demo_url, mode="bn_simple")
print_summary(demo_result)


🌐 Fetching article from: https://www.bbc.com/news/articles/c891jp9j79do
✅ Extracted 903 words from article.
📰 TITLE: Trump says he will sue BBC for at least $1bn over Panorama edit
🔗 URL   : https://www.bbc.com/news/articles/c891jp9j79do
🏷️  Category: Politics
😊 Sentiment: Negative
--------------------------------------------------------------------------------
📄 SUMMARY:

যুক্তরাষ্ট্রের সাবেক প্রেসিডেন্ট ডোনাল্ড ট্রাম্প বিবিসি-র বিরুদ্ধে বিশাল অংকের মামলা করার হুমকি দিয়েছেন। বিবিসি তাদের 'প্যানোরামা' অনুষ্ঠানে ট্রাম্পের একটি গুরুত্বপূর্ণ বক্তৃতার অংশ ভুলভাবে সম্পাদনা করেছিল, যা নিয়ে এই বিতর্ক। বিবিসি তাদের ভুলের জন্য ক্ষমা চেয়েছে, তবে ট্রাম্পকে কোনো আর্থিক ক্ষতিপূরণ দিতে রাজি হয়নি।

বিবিসি জানুয়ারি ৬, ২০২১-এর ঘটনা নিয়ে ট্রাম্পের দেওয়া একটি বক্তৃতা এমনভাবে সম্পাদনা করেছিল, যাতে ভুল করে মনে হয়েছিল ট্রাম্প সরাসরি হিংসাত্মক পদক্ষেপ নেওয়ার আহ্বান করছেন। এই ভুল সম্পাদনার জন্য ট্রাম্প মনে করছেন তিনি প্রতারিত হয়েছেন এবং তার মুখ থেকে বের হওয়া কথাগুলো বিকৃত করা হয়েছে।

ট্রাম্প জানিয

In [6]:
# ==================================
# 6. Multi-URL Daily Digest
# ==================================

def summarize_multiple_urls(urls: List[str], mode: str = "bn_simple") -> List[Dict[str, Any]]:
    results = []
    for u in urls:
        print("\n" + "#" * 80)
        print("Processing:", u)
        results.append(summarize_news_url(u, mode=mode))
    return results


def build_daily_digest(results: List[Dict[str, Any]], mode: str = "bn_simple") -> str:
    """Ask Gemini to build a concise daily briefing from multiple article summaries."""
    items = []
    for r in results:
        items.append({
            "title": r.get("title", ""),
            "category": r.get("category", ""),
            "summary": r.get("summary", ""),
            "sentiment": r.get("sentiment", ""),
        })

    prompt_obj = {
        "instruction": f"Create a daily news briefing in language mode: {mode}.",
        "items": items,
    }

    prompt = json.dumps(prompt_obj, ensure_ascii=False)

    try:
        response = model.generate_content(prompt)
        return response.text or "(no response)"
    except Exception as e:
        return f"Model error while building daily digest: {e}"


def print_daily_digest(digest_text: str) -> None:
    print("\n" + "=" * 80)
    print("📰 DAILY BRIEFING")
    print("=" * 80)
    print(digest_text)
    print("=" * 80)

print("✅ Daily digest helpers ready.")


✅ Daily digest helpers ready.


In [7]:
# Example usage for Daily Digest (you can add your own URLs)

digest_urls = [
    "https://www.bbc.com/news/articles/c891jp9j79do",
    # Add more news URLs here if you want
]

digest_results = summarize_multiple_urls(digest_urls, mode="bn_simple")
daily_digest_text = build_daily_digest(digest_results, mode="bn_simple")
print_daily_digest(daily_digest_text)



################################################################################
Processing: https://www.bbc.com/news/articles/c891jp9j79do
🌐 Fetching article from: https://www.bbc.com/news/articles/c891jp9j79do
✅ Extracted 903 words from article.

📰 DAILY BRIEFING
আজকের দৈনিক সংবাদ ব্রিফিং-এ আপনাকে স্বাগতম।

**আজকের প্রধান খবর:**

*   **রাজনীতি:** ডোনাল্ড ট্রাম্প বিবিসির বিরুদ্ধে অন্তত ১ বিলিয়ন ডলার ক্ষতিপূরণের মামলা করবেন বলে জানিয়েছেন। ট্রাম্পের অভিযোগ, বিবিসি-র 'প্যানোরামা' অনুষ্ঠানে তার ২০২১ সালের ৬ জানুয়ারির ভাষণ এমনভাবে সম্পাদনা করা হয়েছিল, যা দেখে মনে হতে পারে তিনি সহিংসতার জন্য উসকানি দিয়েছেন। বিবিসি এই অনিচ্ছাকৃত ভুলের জন্য দুঃখ প্রকাশ করলেও, তারা ট্রাম্পকে কোনো আর্থিক ক্ষতিপূরণ দিতে রাজি নয়। ট্রাম্পের দাবি, বিবিসি 'প্রতারণা' করেছে এবং তার কথা বদলে দিয়েছে। এই ঘটনার জেরে বিবিসির মহাপরিচালক এবং সংবাদ প্রধান পদত্যাগ করেছেন।

এই ছিল আজকের সংক্ষিপ্ত সংবাদ। আরও খবর জানতে আমাদের সাথে থাকুন।


In [8]:
# ==================================
# 7. Prompt Experiments (for report / analysis)
# ==================================

def experiment_prompts(article_text: str, url: str) -> Dict[str, str]:
    """Compare different prompt styles for the same article."""
    article_trimmed = article_text[:MAX_ARTICLE_CHARS]

    base = f"URL: {url}\n\nArticle:\n{article_trimmed}"

    prompt_variants = {
        "short_bullets": "Summarize in Bangla using ONLY 5 bullet points.",
        "detailed_paragraphs": "Summarize in Bangla using 2 paragraphs and then 3 bullet points.",
        "json_structured": "Summarize in Bangla and respond ONLY in JSON with keys: summary, bullets, category.",
    }

    outputs: Dict[str, str] = {}

    for name, instr in prompt_variants.items():
        print("\nRunning variant:", name)
        full_prompt = instr + "\n\n" + base
        try:
            resp = model.generate_content(full_prompt)
            outputs[name] = resp.text or ""
        except Exception as e:
            outputs[name] = f"Model error: {e}"

    return outputs


# Example experiment on a single article (you can comment this out if not needed)
exp_title, exp_article = fetch_article_from_url("https://www.bbc.com/news/articles/c891jp9j79do")
exp_outputs = experiment_prompts(exp_article, "https://www.bbc.com/news/articles/c891jp9j79do")

for name, text_out in exp_outputs.items():
    print("\n" + "=" * 80)
    print("VARIANT:", name)
    print("=" * 80)
    print(text_out)


🌐 Fetching article from: https://www.bbc.com/news/articles/c891jp9j79do
✅ Extracted 903 words from article.

Running variant: short_bullets

Running variant: detailed_paragraphs

Running variant: json_structured

VARIANT: short_bullets
এখানে বিবিসি নিবন্ধটির একটি সংক্ষিপ্ত সার বাংলাতে দেওয়া হলো:

*   ডোনাল্ড ট্রাম্প বিবিসি-এর বিরুদ্ধে তার ৬ জানুয়ারি, ২০২১-এর বক্তৃতা ভুলভাবে সম্পাদনা করার জন্য ১ থেকে ৫ বিলিয়ন ডলারের মামলা করার হুমকি দিয়েছেন।
*   বিবিসি অনিচ্ছাকৃতভাবে "সহিংসতার সরাসরি আহ্বান" এর ভুল ধারণা দেওয়ার জন্য ক্ষমা চেয়েছে, তবে প্রেসিডেন্ট ট্রাম্পকে কোনো আর্থিক ক্ষতিপূরণ দিতে অস্বীকার করেছে।
*   বিবিসি-এর 'প্যানোরামা' অনুষ্ঠানে ট্রাম্পের দুটি ভিন্ন অংশের বক্তব্যকে একত্রিত করে এমনভাবে দেখানো হয়েছিল, যা তাকে সহিংস কর্মের জন্য সরাসরি আহ্বান জানানো হিসাবে ভুলভাবে উপস্থাপন করে।
*   এই বিতর্কের জেরে বিবিসি-এর মহাপরিচালক ও সংবাদ প্রধান পদত্যাগ করেছেন, এবং এটি প্রতিষ্ঠানটির জন্য একটি "দুঃস্বপ্ন" পরিস্থিতি তৈরি করেছে।
*   ট্রাম্প বলেছেন যে বিবিসি "প্রতারণা" করেছে এবং ভবিষ্যতে যাতে এমন 

In [9]:
# ==================================
# 8. Simple Evaluation Against Human Summary
# ==================================

# You should replace the 'human_summary' text with your own manual summary
# for proper evaluation.

eval_data = [
    {
        "url": "https://www.bbc.com/news/articles/c891jp9j79do",
        "human_summary": "Write your human-made reference summary here.",
    },
]


def run_eval(data_list: List[Dict[str, str]], mode: str = "bn_simple") -> List[Dict[str, Any]]:
    results: List[Dict[str, Any]] = []
    for item in data_list:
        url = item["url"]
        human = item["human_summary"]

        ai_result = summarize_news_url(url, mode=mode)
        ai_summary = ai_result.get("summary", "")

        human_words = len(human.split())
        ai_words = len(ai_summary.split())
        length_ratio = round(ai_words / max(human_words, 1), 2)

        results.append({
            "url": url,
            "human_words": human_words,
            "ai_words": ai_words,
            "length_ratio": length_ratio,
            "human_summary": human,
            "ai_summary": ai_summary,
        })
    return results


evaluations = run_eval(eval_data, mode="bn_simple")

for r in evaluations:
    print("\n" + "=" * 80)
    print("URL:", r["url"])
    print("Human words:", r["human_words"])
    print("AI words:", r["ai_words"])
    print("Length ratio (AI/Human):", r["length_ratio"])
    print("\nHuman Summary:\n", r["human_summary"])
    print("\nAI Summary:\n", r["ai_summary"])


🌐 Fetching article from: https://www.bbc.com/news/articles/c891jp9j79do
✅ Extracted 903 words from article.

URL: https://www.bbc.com/news/articles/c891jp9j79do
Human words: 6
AI words: 164
Length ratio (AI/Human): 27.33

Human Summary:
 Write your human-made reference summary here.

AI Summary:
 ডোনাল্ড ট্রাম্প বলেছেন যে তিনি বিবিসি-র বিরুদ্ধে আইনি ব্যবস্থা নেবেন। কারণ বিবিসি তার একটি ভাষণ এমনভাবে সম্পাদনা করেছে, যা ভুল বার্তা দিয়েছে। যদিও বিবিসি এই ভুল সম্পাদনার জন্য ক্ষমা চেয়েছে, তবে তারা কোনো ক্ষতিপূরণ দিতে রাজি হয়নি। ট্রাম্প এর জন্য ১ বিলিয়ন থেকে ৫ বিলিয়ন ডলার ক্ষতিপূরণ দাবি করছেন।

ট্রাম্পের অভিযোগ, বিবিসি তার জানুয়ারি ৬, ২০২১ সালের ভাষণটি এমনভাবে কেটেছে যেন মনে হয় তিনি সহিংস পদক্ষেপের জন্য সরাসরি ডাক দিয়েছেন। বিবিসি অবশ্য স্বীকার করেছে যে তাদের সম্পাদনার কারণে অনিচ্ছাকৃতভাবে এমন ভুল ধারণা তৈরি হয়েছিল এবং তারা এটি আর প্রচার করবে না। তারা ক্ষমা চাইলেও ক্ষতিপূরণের দাবি প্রত্যাখ্যান করেছে।

ট্রাম্প বলেছেন, তিনি মামলা করবেন কারণ বিবিসি তার কথা বিকৃত করেছে। এই পরিস্থিতিতে, প্

## 9. Limitations & Future Work

### Limitations
- Relies only on article text; no external fact-checking.
- Paywalled or heavily JavaScript-based sites may not scrape correctly.
- Very short or noisy articles may not produce good summaries.
- AI may carry over bias from the original news source.

### Future Work
- Add custom scrapers for Bangla news websites.
- Integrate external fact-checking or cross-source verification.
- Build a browser extension ("Summarize this article").
- Create a mobile app or Telegram bot around this pipeline.
- Add ROUGE/BERTScore metrics for a more robust quantitative evaluation.
- Allow user to control summary length and tone from a simple UI.
